In [4]:
u = lambda x: np.exp(-800*(x-0.4)**2) + 0.25 * np.exp(-40*(x-0.8)**2)
import sympy as sp

x = sp.symbols('x')
u_sym = sp.exp(-800*(x-0.4)**2) + 0.25 * sp.exp(-40*(x-0.8)**2)

In [5]:
u_double_diff = sp.diff(u_sym, x, 2)
u_sym-u_double_diff

-1600.0*(x - 0.8)**2*exp(-40*(x - 0.8)**2) - 2560000*(x - 0.4)**2*exp(-800*(x - 0.4)**2) + 1601*exp(-800*(x - 0.4)**2) + 20.25*exp(-40*(x - 0.8)**2)

In [6]:
u_sym.subs(x,0)

1.90546629862822e-12

In [7]:
u_sym.subs(x,1)

0.0504741294986639

*b)*

In [8]:
import numpy as np

def compute_error_decrease(uc,uf,VX,EToV):
    
    N = len(EToV[:,0])
    err = np.zeros(N)

    for i, pair in enumerate(EToV):

        j = pair[0]
        k = pair[0]

        xi   = VX[j]
        xip1 = VX[k]
        xih  =  xi + (xip1 - xi)/2

        ui   = uc[j]
        uip1 = uc[k]
                

        a = ( ui + 2 * uih - uip1 ) / (xi - xip1)

        a_a1 = ( - ui + 2 * uih - uip1 ) / (xi - xip1)
        a_a2 = - a_a1     
        b_b1 = a_a2 * xi
        b_b2 = a_a1*xip1

        int1 = (a_a1**2/3) * (xih**3 - xi**3) + (b_b1)**2 * (xih - xi) + (a_a1)*(b_b1) * (xih**2 - xi**2)
        int2 = (a_a2**2/3) * (xip1**3 - xih**3) + (b_b2)**2 * (xip1 - xih) + (a_a2)*(b_b2) * (xip1**2 - xih**2)
        # if int1 < 0 or int2 < 0:
        #     print('a_a1',a_a1)
        #     print('a_a2',a_a2)
        #     print('b_b1',b_b1)
        #     print('b_b2',b_b2)
        #     print('xi',xi)
        #     print('xip1',xip1)
        #     return False
            
        err[i] = np.sqrt(int1 + int2)

    return err

*e)*

In [55]:
import numpy as np
from scipy.sparse import csr_matrix
K = lambda h, psi, eps: np.array([[eps/h + psi/2, -eps/h + psi/2], [-eps/h - psi/2, eps/h - psi/2]])

def GlobalAssemblyOld(x,eps,psi,c,d):
    M = len(x)
    nnzmax = 4 * M
    ii = np.ones(nnzmax, dtype=int)
    jj = np.ones(nnzmax, dtype=int)
    ss = np.zeros(nnzmax)
    b = np.zeros(M)
    count = 0
    for i in range(M - 1):
        h = x[i+1] - x[i]
        b[i] += h/2
        b[i+1] += h/2

        Ki = K(h,psi,eps)
        print(Ki)

        ii[count:count + 4] = [i, i, i + 1, i + 1]
        jj[count:count + 4] = [i, i + 1, i + 1, i]
        ss[count:count + 4] = [
        Ki[0, 0],
        Ki[0, 1],
        Ki[1, 1],
        Ki[1, 0]
        ]
        count += 4
    
    A = csr_matrix((ss[:count], (ii[:count], jj[:count])), shape=(M, M))

    # Boundary conditions
    # b[0] = c
    # b[1] -= A[0,1]*c

    # A[0,0] = 1
    # A[0,1] = 0
    # A[1,0] = 0
    
    # b[M-1] = d
    # b[M-2] -= A[M-1,M-2]*d

    # A[M-1,M-1] = 1
    # A[M-1,M-2] = 0
    # A[M-2,M-1] = 0

    return A, b

In [59]:
u = lambda x,epsinv,psi: 1/psi * (1+(np.exp(psi*epsinv)-1)*x - np.exp(x*psi*epsinv))/(np.exp(psi*epsinv)-1)

A_Old,b_Old = GlobalAssemblyOld(x=np.linspace(0,1,5),eps=1,psi=1,c=0,d=0)
print(A_Old.todense())


[[ 4.5 -3.5]
 [-4.5  3.5]]
[[ 4.5 -3.5]
 [-4.5  3.5]]
[[ 4.5 -3.5]
 [-4.5  3.5]]
[[ 4.5 -3.5]
 [-4.5  3.5]]
[[ 4.5 -3.5  0.   0.   0. ]
 [-4.5  8.  -3.5  0.   0. ]
 [ 0.  -4.5  8.  -3.5  0. ]
 [ 0.   0.  -4.5  8.  -3.5]
 [ 0.   0.   0.  -4.5  3.5]]


# New version

In [69]:
import numpy as np
from scipy.sparse import csr_matrix
K = lambda h, psi, eps: np.array([[eps/h + psi/2, -eps/h + psi/2], [-eps/h - psi/2, eps/h - psi/2]])

def GlobalAssembly(VX,EToV,eps,psi,c,d):
    M = len(VX)
    nnzmax = 4 * M
    ii = np.ones(nnzmax, dtype=int)
    jj = np.ones(nnzmax, dtype=int)
    ss = np.zeros(nnzmax)
    b = np.zeros(M)
    count = 0

    for pair in EToV:
        i = pair[0]
        j = pair[1]

        h = VX[j] - VX[i]
        print(h)
        b[i] += h/2
        b[j] += h/2

        Ki = K(h,psi,eps)
        print(Ki)

        ii[count:count + 4] = [i, i, j, j]
        jj[count:count + 4] = [i, j, j, i]
        ss[count:count + 4] = [
        Ki[0, 0],
        Ki[0, 1],
        Ki[1, 1],
        Ki[1, 0]
        ]
        count += 4
    
    A = csr_matrix((ss[:count], (ii[:count], jj[:count])), shape=(M, M))

    # Boundary conditions
    # b[0] = c
    # b[1] -= A[0,1]*c

    # A[0,0] = 1
    # A[0,1] = 0
    # A[1,0] = 0
    
    # b[M-1] = d
    # b[M-2] -= A[M-1,M-2]*d

    # A[M-1,M-1] = 1
    # A[M-1,M-2] = 0
    # A[M-2,M-1] = 0

    return A, b

In [64]:
EToV = np.array([[0,3],[2,4],[3,2],[4,1]])
VX = np.array([0,1,0.5,0.25,0.75])
print(EToV)
print(VX)


[[0 3]
 [2 4]
 [3 2]
 [4 1]]
[0.   1.   0.5  0.25 0.75]


In [70]:
u = lambda x,epsinv,psi: 1/psi * (1+(np.exp(psi*epsinv)-1)*x - np.exp(x*psi*epsinv))/(np.exp(psi*epsinv)-1)
x = np.linspace(0,1,5)

A,b = GlobalAssembly(VX=VX,EToV=EToV,eps=1,psi=1,c=0,d=0)

print(A.todense(),b)
print(A_Old.todense(),b_Old)

0.25
[[ 4.5 -3.5]
 [-4.5  3.5]]
0.25
[[ 4.5 -3.5]
 [-4.5  3.5]]
0.25
[[ 4.5 -3.5]
 [-4.5  3.5]]
0.25
[[ 4.5 -3.5]
 [-4.5  3.5]]
[[ 4.5  0.   0.  -3.5  0. ]
 [ 0.   3.5  0.   0.  -4.5]
 [ 0.   0.   8.  -4.5 -3.5]
 [-4.5  0.  -3.5  8.   0. ]
 [ 0.  -3.5 -4.5  0.   8. ]] [0.125 0.125 0.25  0.25  0.25 ]
[[ 4.5 -3.5  0.   0.   0. ]
 [-4.5  8.  -3.5  0.   0. ]
 [ 0.  -4.5  8.  -3.5  0. ]
 [ 0.   0.  -4.5  8.  -3.5]
 [ 0.   0.   0.  -4.5  3.5]] [0.125 0.25  0.25  0.25  0.125]


In [71]:
print('Old')
GlobalAssemblyOld(x=np.linspace(0,1,5),eps=1,psi=1,c=0,d=0)
print('New')
GlobalAssembly(VX=VX,EToV=EToV,eps=1,psi=1,c=0,d=0)

Old
[[ 4.5 -3.5]
 [-4.5  3.5]]
[[ 4.5 -3.5]
 [-4.5  3.5]]
[[ 4.5 -3.5]
 [-4.5  3.5]]
[[ 4.5 -3.5]
 [-4.5  3.5]]
New
0.25
[[ 4.5 -3.5]
 [-4.5  3.5]]
0.25
[[ 4.5 -3.5]
 [-4.5  3.5]]
0.25
[[ 4.5 -3.5]
 [-4.5  3.5]]
0.25
[[ 4.5 -3.5]
 [-4.5  3.5]]


(<5x5 sparse matrix of type '<class 'numpy.float64'>'
 	with 13 stored elements in Compressed Sparse Row format>,
 array([0.125, 0.125, 0.25 , 0.25 , 0.25 ]))